In [1]:
import asyncio
import dagshub
from dagshub.common.api.repo import RepoAPI
from dagshub.data_engine.datasources import get_datasource
from io import BytesIO
import json
from pathlib import Path
from PIL import Image
import requests
import time

import plotly.express as px
import plotly.graph_objects as go

token = '9379eb5d9f33527968c7253f77074ac56d67d9e4'
headers = {"Authorization": f"Bearer {token}"}

In [2]:
repo_id, ds_id = 'Dean/COCO_1K', 'COCO_1K'
#repo_id, ds_id = 'OperationSavta/SavtaDepth', 'NYU_Depth_Processed'
#repo_id, ds_id = 'Omdena/TriesteItalyChapter_MappingSeagrassMeadows', 'Seagrass_ROI_Winter'
repo = RepoAPI(repo_id)

repo_info = repo.get_repo_info()

path_list = {}
for path in repo.list_path(''):
    path_list.setdefault(path.versioning, []).append(path)

ds = get_datasource(repo_id, ds_id)
query_all = ds.all()
df = query_all.dataframe
print(ds)

Output()

Datasource COCO_1K
	Repo: Dean/COCO_1K, path: repo://Dean/COCO_1K/main:data/images
	DatasourceQuery(as_of=None, select=None, filter=Query: empty, order_by=None)
	Fields:
		media type (STRING)
		height (INTEGER)
		width (INTEGER)
		annotation (BLOB) with tags: {'annotation'}
		custom_metadata (STRING)
		size (INTEGER)
		categories (STRING)
		valid_datapoint (BOOLEAN)
		year (INTEGER)
		split (STRING)



In [3]:
async def load_annotations(query):
    start_time = time.time()
    print(f"loading annotations for {query}")
    annotations = query.get_blob_fields(
        'annotation', 
        load_into_memory=True,
        cache_on_disk=False,
        num_proc=32,
    )
    print(f"loaded annotations for {query} in {time.time()-start_time} seconds")
    return annotations

In [5]:
task = asyncio.create_task(load_annotations(query_all))

loading annotations for QueryResult of datasource COCO_1K with 1199 datapoint(s)


Output()

In [4]:
print('yes')

yes


In [10]:
result = await task

In [17]:
result.dataframe.iloc[0]

path                                               train/000000005105.jpg
datapoint_id                                                     44457250
dagshub_download_url    https://dagshub.com/api/v1/repos/Dean/COCO_1K/...
annotation              b'{"id":6368,"data":{"image":"https://dagshub....
categories                bench, dog, sports ball, person, car, surfboard
height                                                                480
media type                                                     image/jpeg
size                                                               127731
split                                                               train
valid_datapoint                                                      True
width                                                                 640
year                                                                 2017
Name: 0, dtype: object

In [26]:
anno = {
    i : json.loads(result.dataframe.iloc[i].annotation)
    for i in range(len(result.dataframe))
}

In [29]:
list(anno[0].keys())

['id',
 'data',
 'meta',
 'created_at',
 'updated_at',
 'is_labeled',
 'project',
 'annotations']

In [39]:
anno[0].get('annotations')

[{'completed_by': 5,
  'result': [{'from_name': 'label',
    'id': '27a0c20f2a',
    'image_rotation': 0,
    'original_height': 480,
    'original_width': 640,
    'to_name': 'image',
    'type': 'polygonlabels',
    'value': {'closed': True,
     'points': [[44.7875, 77.1979],
      [44.987500000000004, 76.58749999999999],
      [45.2156, 75.9396],
      [45.5297, 75.4437],
      [46.0734, 75.2146],
      [46.273399999999995, 74.9875],
      [46.8172, 74.3771],
      [47.3031, 74.1104],
      [48.4187, 74.1104],
      [49.2188, 74.49170000000001],
      [50.7047, 75.71039999999999],
      [51.1344, 76.2438],
      [52.7359, 77.8458],
      [53.8797, 78.9125],
      [57.1094, 80.3625],
      [58.62500000000001, 80.0958],
      [60.398399999999995, 80.0958],
      [61.712500000000006, 79.9042],
      [65.9156, 79.9417],
      [69.6047, 80.1708],
      [71.17660000000001, 79.8271],
      [74.6078, 79.2563],
      [76.2375, 79.1417],
      [78.2672, 78.7604],
      [79.58279999999999, 78

In [42]:
list(anno[0].get('annotations')[0].keys())

['completed_by', 'result', 'ground_truth']

In [43]:
anno[0].get('annotations')[0].get('completed_by')

5

In [45]:
anno[0].get('annotations')[0].get('ground_truth')

False

In [44]:
anno[0].get('annotations')[0].get('result')

[{'from_name': 'label',
  'id': '27a0c20f2a',
  'image_rotation': 0,
  'original_height': 480,
  'original_width': 640,
  'to_name': 'image',
  'type': 'polygonlabels',
  'value': {'closed': True,
   'points': [[44.7875, 77.1979],
    [44.987500000000004, 76.58749999999999],
    [45.2156, 75.9396],
    [45.5297, 75.4437],
    [46.0734, 75.2146],
    [46.273399999999995, 74.9875],
    [46.8172, 74.3771],
    [47.3031, 74.1104],
    [48.4187, 74.1104],
    [49.2188, 74.49170000000001],
    [50.7047, 75.71039999999999],
    [51.1344, 76.2438],
    [52.7359, 77.8458],
    [53.8797, 78.9125],
    [57.1094, 80.3625],
    [58.62500000000001, 80.0958],
    [60.398399999999995, 80.0958],
    [61.712500000000006, 79.9042],
    [65.9156, 79.9417],
    [69.6047, 80.1708],
    [71.17660000000001, 79.8271],
    [74.6078, 79.2563],
    [76.2375, 79.1417],
    [78.2672, 78.7604],
    [79.58279999999999, 78.3021],
    [80.35470000000001, 78.1896],
    [79.8406, 79.4458],
    [78.6391, 79.4458],
    [71

In [47]:
len( anno[0].get('annotations')[0].get('result') )

15

In [48]:
anno[0].get('annotations')[0].get('result')[0]

{'from_name': 'label',
 'id': '27a0c20f2a',
 'image_rotation': 0,
 'original_height': 480,
 'original_width': 640,
 'to_name': 'image',
 'type': 'polygonlabels',
 'value': {'closed': True,
  'points': [[44.7875, 77.1979],
   [44.987500000000004, 76.58749999999999],
   [45.2156, 75.9396],
   [45.5297, 75.4437],
   [46.0734, 75.2146],
   [46.273399999999995, 74.9875],
   [46.8172, 74.3771],
   [47.3031, 74.1104],
   [48.4187, 74.1104],
   [49.2188, 74.49170000000001],
   [50.7047, 75.71039999999999],
   [51.1344, 76.2438],
   [52.7359, 77.8458],
   [53.8797, 78.9125],
   [57.1094, 80.3625],
   [58.62500000000001, 80.0958],
   [60.398399999999995, 80.0958],
   [61.712500000000006, 79.9042],
   [65.9156, 79.9417],
   [69.6047, 80.1708],
   [71.17660000000001, 79.8271],
   [74.6078, 79.2563],
   [76.2375, 79.1417],
   [78.2672, 78.7604],
   [79.58279999999999, 78.3021],
   [80.35470000000001, 78.1896],
   [79.8406, 79.4458],
   [78.6391, 79.4458],
   [71.9484, 79.86670000000001],
   [70.604

In [49]:
list(anno[0].get('annotations')[0].get('result')[0].keys())

['from_name',
 'id',
 'image_rotation',
 'original_height',
 'original_width',
 'to_name',
 'type',
 'value']

In [52]:
anno[0].get('annotations')[0].get('result')[0].get('from_name')

'label'

In [53]:
anno[0].get('annotations')[0].get('result')[0].get('id')

'27a0c20f2a'

In [54]:
anno[0].get('annotations')[0].get('result')[0].get('image_rotation')

0

In [55]:
anno[0].get('annotations')[0].get('result')[0].get('original_height')

480

In [56]:
anno[0].get('annotations')[0].get('result')[0].get('original_width')

640

In [57]:
anno[0].get('annotations')[0].get('result')[0].get('to_name')

'image'

In [58]:
anno[0].get('annotations')[0].get('result')[0].get('type')

'polygonlabels'

In [59]:
anno[0].get('annotations')[0].get('result')[0].get('value')

{'closed': True,
 'points': [[44.7875, 77.1979],
  [44.987500000000004, 76.58749999999999],
  [45.2156, 75.9396],
  [45.5297, 75.4437],
  [46.0734, 75.2146],
  [46.273399999999995, 74.9875],
  [46.8172, 74.3771],
  [47.3031, 74.1104],
  [48.4187, 74.1104],
  [49.2188, 74.49170000000001],
  [50.7047, 75.71039999999999],
  [51.1344, 76.2438],
  [52.7359, 77.8458],
  [53.8797, 78.9125],
  [57.1094, 80.3625],
  [58.62500000000001, 80.0958],
  [60.398399999999995, 80.0958],
  [61.712500000000006, 79.9042],
  [65.9156, 79.9417],
  [69.6047, 80.1708],
  [71.17660000000001, 79.8271],
  [74.6078, 79.2563],
  [76.2375, 79.1417],
  [78.2672, 78.7604],
  [79.58279999999999, 78.3021],
  [80.35470000000001, 78.1896],
  [79.8406, 79.4458],
  [78.6391, 79.4458],
  [71.9484, 79.86670000000001],
  [70.6047, 80.4],
  [70.8625, 81.6583],
  [72.0625, 81.5063],
  [72.9203, 81.5063],
  [73.75, 81.69579999999999],
  [74.2078, 81.69579999999999],
  [75.4938, 82.53540000000001],
  [78.3813, 82.53540000000001],


In [ ]:
import itertools
from collections import Counter
counts = Counter(itertools.chain(*(x.split(', ') for x in df['categories'])))
px.bar(x=list(counts.keys()), y=list(counts.values()))

In [81]:
!pip install -e /home/zndr27/repos/dtale

Obtaining file:///home/zndr27/repos/dtale
  Preparing metadata (setup.py) ... done
  Using cached dash_bootstrap_components-1.3.1-py3-none-any.whl (219 kB)
  Using cached Flask-2.2.5-py3-none-any.whl (101 kB)
  Attempting uninstall: Flask
    Found existing installation: Flask 3.0.3
    Uninstalling Flask-3.0.3:
      Successfully uninstalled Flask-3.0.3
  Attempting uninstall: dash-bootstrap-components
    Found existing installation: dash-bootstrap-components 1.6.0
    Uninstalling dash-bootstrap-components-1.6.0:
      Successfully uninstalled dash-bootstrap-components-1.6.0
  Attempting uninstall: dtale
    Found existing installation: dtale 3.12.0
    Uninstalling dtale-3.12.0:
      Successfully uninstalled dtale-3.12.0
  Running setup.py develop for dtale

[notice] A new release of pip is available: 23.1.2 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [8]:
from dash import Dash
from dtale.app import build_app
from dtale.views import startup
dtale_app, dtale_dash = build_app(reaper_on=False)
instance = startup(data=df, ignore_duplicate=True)

In [9]:
type(dtale_app)

dtale.app.DtaleFlask

In [10]:
type(dtale_dash)

dtale.dash_application.views.DtaleDash

In [11]:
dtale_dash.layout

Div([Location(id='url', refresh=False), Div(id='popup-content')])